In [ ]:
# File to create full dataset with all variables based on selected entries

In [1]:
#Load packages
import pyreadr
import pandas as pd
import numpy as np
from IPython.display import display 

In [2]:
# Load created function
from utils import add_latest_entry_before_date, add_latest_diagnosis_entry

In [3]:
# Read-in dataframe generated in R: "Approach including Trend of HbA1c"
# input_data_trend has been uploaded --> 227k entries 
input = pyreadr.read_r("/home/jupyter-niclas/input_data_trend_NEW.rds")
input_data_trend = next(iter(input.values()))
# Renaming of column
input_data_trend = input_data_trend.rename(columns={'val': 'val_HbA1c'})
display(input_data_trend)

,idp,value_mean,delta_last,relative_change,dat,val_HbA1c
0,0000072333d55312fe62d5b5458a22efa4257e27,5.97,0.7,0.1607,20230427,6.5
1,0000150454aa8670459408f45e453242f7bcde9c,5.80,0.4,0.0517,20230215,6.1
2,0000214c9856e03de46a8f927fc1e08b7c943578,6.04,-0.3,0.0000,20221207,5.9
3,000058a85618eb0c0d59f3e9056372e87e5d426e,5.40,0.6,0.0943,20201030,5.8
4,000085fdce1bb6d9e93904e0c632609e1c59085f,6.03,0.0,0.1250,20220328,6.3
...,...,...,...,...,...,...
227362,fffe931b53eb160df80efc3d076c7a39c57b2780,6.17,0.3,0.0161,20110516,6.3
227363,fffeae900480853f4b01e124674bbe67e283c62f,5.89,-0.1,0.0714,20230426,6.0
227364,fffeffb3cbeb5a83dfb6a6e4cec485ff76fb79de,6.07,-0.2,0.0169,20220914,6.0
227365,ffff8531e62f5d66787f60582f9d439732650d8e,6.13,-0.3,0.0164,20160927,6.2


In [4]:
# Adding column sexe to input_data_trend according to data_poblacio
poblacio = pyreadr.read_r("/home/dadesSP_berta75/taulesSIDIAP/taulesSIDIAP/ERC_ERC_DM_entregable_poblacio_20240409_202620.rds")
data_poblacio = next(iter(poblacio.values()))
input_data_trend = input_data_trend.merge(data_poblacio[['idp', 'sexe']], on='idp', how='left')
# Check new column for number of NAs
sexe_NA = input_data_trend['sexe'].isna().sum()
print (sexe_NA)

0


In [5]:
# Adding column birth_date to input_data_trend according to dnaix from data_poblacio
input_data_trend = input_data_trend.merge(data_poblacio[['idp', 'dnaix']], on='idp', how='left')
# Check new column for number of NAs
dnaix_NA = input_data_trend['dnaix'].isna().sum()
print (dnaix_NA)

0


In [6]:
# Using onset-date and birth date to determine age at t0
# Convert to datetime format
input_data_trend['dat'] = pd.to_datetime(input_data_trend['dat'].astype(str), format='%Y%m%d')
input_data_trend['dnaix'] = pd.to_datetime(input_data_trend['dnaix'].astype(str), format='%Y%m%d')
# Calculate age (in years)
input_data_trend['age'] = (input_data_trend['dat'] - input_data_trend['dnaix']).dt.days // 365
# Exclude individuals younger than 18
input_data_trend = input_data_trend[input_data_trend['age'] >= 18]
# Check for NAs in age
age_NA = input_data_trend['age'].isna().sum()
print(f"Number of missing age entries: {age_NA}")
display (input_data_trend)

Number of missing age entries: 0


,idp,value_mean,delta_last,relative_change,dat,val_HbA1c,sexe,dnaix,age
0,0000072333d55312fe62d5b5458a22efa4257e27,5.97,0.7,0.1607,2023-04-27,6.5,D,1949-09-01,73
1,0000150454aa8670459408f45e453242f7bcde9c,5.80,0.4,0.0517,2023-02-15,6.1,H,1962-02-01,61
2,0000214c9856e03de46a8f927fc1e08b7c943578,6.04,-0.3,0.0000,2022-12-07,5.9,H,1947-06-01,75
3,000058a85618eb0c0d59f3e9056372e87e5d426e,5.40,0.6,0.0943,2020-10-30,5.8,H,1949-02-01,71
4,000085fdce1bb6d9e93904e0c632609e1c59085f,6.03,0.0,0.1250,2022-03-28,6.3,D,1930-08-01,91
...,...,...,...,...,...,...,...,...,...
227362,fffe931b53eb160df80efc3d076c7a39c57b2780,6.17,0.3,0.0161,2011-05-16,6.3,D,1933-03-01,78
227363,fffeae900480853f4b01e124674bbe67e283c62f,5.89,-0.1,0.0714,2023-04-26,6.0,D,1951-05-01,72
227364,fffeffb3cbeb5a83dfb6a6e4cec485ff76fb79de,6.07,-0.2,0.0169,2022-09-14,6.0,H,1954-05-01,68
227365,ffff8531e62f5d66787f60582f9d439732650d8e,6.13,-0.3,0.0164,2016-09-27,6.2,H,1934-03-01,82


In [7]:
# Adding column smoking status from tabaquisme
tabaquisme = pyreadr.read_r("/home/dadesSP_berta75/taulesSIDIAP/taulesSIDIAP/ERC_ERC_DM_entregable_tabaquisme_20240409_202620.rds")
data_tabaquisme = next(iter(tabaquisme.values()))
# Converting code into smoking status
smoking_mapping = {
    0: "No smoker",
    1: "Smoker",
    2: "Ex-smoker"
}
data_tabaquisme['val'] = data_tabaquisme['val'].replace(smoking_mapping)
data_tabaquisme.rename(columns={'val': 'smoking_status'}, inplace=True)

In [8]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_tabaquisme,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='smoking_status'   
)

Number of missing values in 'smoking_status': 2482


In [9]:
# Adding column alcohol risk consumption from variables_cliniques --> subset ALRIS generated in R
alris = pyreadr.read_r("Data_preparation/data_ALRIS.rds")
data_alris = next(iter(alris.values()))

In [10]:
# Converting code into smoking status
alc_ris_mapping = {
    0: "No risk consumption",
    1: "Low risk consumption",
    2: "High risk consumption"
}
data_alris['val'] = data_alris['val'].replace(alc_ris_mapping)

In [11]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_alris,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='alcohol_risk_consumption'
)

Number of missing values in 'alcohol_risk_consumption': 8001


In [12]:
# Adding MEDEA index from variables_socioeconomiques
var_socioeco = pyreadr.read_r("/home/dadesSP_berta75/taulesSIDIAP/taulesSIDIAP/ERC_ERC_DM_entregable_variables_socioeconomiques_20240409_202620.rds")
data_var_socioeco = next(iter(var_socioeco.values()))
# Add both columns to dataframe
input_data_trend = input_data_trend.merge(data_var_socioeco[['idp', 'ruralitat', 'qmedea']], on='idp', how='left')
# Check new column for number of NAs
# Convert empty strings or spaces to NaN
input_data_trend['ruralitat'] = input_data_trend['ruralitat'].replace(r'^\s*$', np.nan, regex=True)
ruralitat_NA = input_data_trend['ruralitat'].isna().sum()
print (ruralitat_NA)
# Check new column for number of NAs
# Convert empty strings or spaces to NaN
input_data_trend['qmedea'] = input_data_trend['qmedea'].replace(r'^\s*$', np.nan, regex=True)
qmedea_NA = input_data_trend['qmedea'].isna().sum()
print (qmedea_NA)
display(input_data_trend)

34763
72007


,idp,value_mean,delta_last,relative_change,dat,val_HbA1c,sexe,dnaix,age,smoking_status,alcohol_risk_consumption,ruralitat,qmedea
0,0000072333d55312fe62d5b5458a22efa4257e27,5.97,0.7,0.1607,2023-04-27,6.5,D,1949-09-01,73,No smoker,No risk consumption,R,NaN
1,0000150454aa8670459408f45e453242f7bcde9c,5.80,0.4,0.0517,2023-02-15,6.1,H,1962-02-01,61,No smoker,No risk consumption,R,NaN
2,0000214c9856e03de46a8f927fc1e08b7c943578,6.04,-0.3,0.0000,2022-12-07,5.9,H,1947-06-01,75,No smoker,Low risk consumption,R,NaN
3,000058a85618eb0c0d59f3e9056372e87e5d426e,5.40,0.6,0.0943,2020-10-30,5.8,H,1949-02-01,71,No smoker,Low risk consumption,U,U1
4,000085fdce1bb6d9e93904e0c632609e1c59085f,6.03,0.0,0.1250,2022-03-28,6.3,D,1930-08-01,91,No smoker,No risk consumption,R,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227312,fffe931b53eb160df80efc3d076c7a39c57b2780,6.17,0.3,0.0161,2011-05-16,6.3,D,1933-03-01,78,No smoker,No risk consumption,NaN,NaN
227313,fffeae900480853f4b01e124674bbe67e283c62f,5.89,-0.1,0.0714,2023-04-26,6.0,D,1951-05-01,72,No smoker,No risk consumption,U,U2
227314,fffeffb3cbeb5a83dfb6a6e4cec485ff76fb79de,6.07,-0.2,0.0169,2022-09-14,6.0,H,1954-05-01,68,Smoker,Low risk consumption,R,NaN
227315,ffff8531e62f5d66787f60582f9d439732650d8e,6.13,-0.3,0.0164,2016-09-27,6.2,H,1934-03-01,82,Ex-smoker,No risk consumption,U,U4


In [13]:
# Start Block Comorbidities
# creating dataframe data_diagnostics
diagnostics = pyreadr.read_r("Data_preparation/data_both_diagnostics.rds")
data_diagnostics = next(iter(diagnostics.values()))

In [14]:
display(data_diagnostics)

,idp,cod,agr,dat,dalta
0,bbbeb529294a4aaa687a841238148a8ddc1455a2,Z91.19,VUL_SOCL,20210227,20210303
1,bbbeb529294a4aaa687a841238148a8ddc1455a2,E11.65,DM,20210227,20210303
2,bbbeb529294a4aaa687a841238148a8ddc1455a2,I10,HTA,20210227,20210303
3,0857c87ecb608ffe2708b0e56a53a7cbcecc6b1f,I48.91,ARRITMIA,20140916,20140917
4,adfd8992dcfd2cccccf3904c1fc71c4a498abfd4,Z96.652,ARTRITIS,20180408,20180410
...,...,...,...,...,...
25382163,285f3d1b5266fb371fe9864bf41393595c74ce7f,I21.3,CAR_ISQ,20160725,NaN
25382164,bb6faa23f14f284393ddfac1be460b19bd2cd2af,I21.9,CAR_ISQ,20220107,NaN
25382165,199a27024c77d06c6ac76a6afe9ae26c735841d6,I21.3,CAR_ISQ,20120429,NaN
25382166,114fce05d03f1aace600fa15fdae2727c68292a3,I21.3,CAR_ISQ,20120201,NaN


In [15]:
#hypertension
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I10', 'I11', 'I12', 'I13', 'I14', 'I15'],
    reference_date_col='dat',
    result_col_name='hypertension'
)

Number of missing values in 'hypertension': 141015


In [16]:
# dyslipidemia
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['E78'],
    reference_date_col='dat',
    result_col_name='dyslipidemia'
)

Number of missing values in 'dyslipidemia': 145965


In [17]:
# Heart failure
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I50'],
    reference_date_col='dat',
    result_col_name='heart_failure'
)

Number of missing values in 'heart_failure': 211107


In [18]:
# Peripheral artery disease
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    fixed_codes=['I73.9'],
    reference_date_col='dat',
    result_col_name='peripheral_artery_disease'
)

Number of missing values in 'peripheral_artey_disease': 220241


In [19]:
# stroke
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I61', 'I62', 'I63'],
    reference_date_col='dat',
    result_col_name='stroke'
)

Number of missing values in 'stroke': 214849


In [20]:
# ischemic heart disease
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I20', 'I21', 'I22', 'I23','I24', 'I25'],
    reference_date_col='dat',
    result_col_name='ischemic_heart_disease'
)

Number of missing values in 'ischemic_heart_disease': 205914


In [21]:
# Chronic kidney disease
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['N18', 'I12', 'I13'],
    reference_date_col='dat',
    result_col_name='chronic_kidney_disease'
)

Number of missing values in 'chronic_kidney_disease': 195762


In [22]:
#END section comorbidities
display(input_data_trend)

,idp,value_mean,delta_last,relative_change,dat,val_HbA1c,sexe,dnaix,age,smoking_status,alcohol_risk_consumption,ruralitat,qmedea,hypertension,dyslipidemia,heart_failure,peripheral_artey_disease,stroke,ischemic_heart_disease,chronic_kidney_disease
0,0000072333d55312fe62d5b5458a22efa4257e27,5.97,0.7,0.1607,2023-04-27,6.5,D,1949-09-01,73,No smoker,No risk consumption,R,NaN,1,1,NaN,NaN,NaN,NaN,NaN
1,0000150454aa8670459408f45e453242f7bcde9c,5.80,0.4,0.0517,2023-02-15,6.1,H,1962-02-01,61,No smoker,No risk consumption,R,NaN,1,NaN,NaN,NaN,1,NaN,NaN
2,0000214c9856e03de46a8f927fc1e08b7c943578,6.04,-0.3,0.0000,2022-12-07,5.9,H,1947-06-01,75,No smoker,Low risk consumption,R,NaN,NaN,1,NaN,NaN,NaN,1,NaN
3,000058a85618eb0c0d59f3e9056372e87e5d426e,5.40,0.6,0.0943,2020-10-30,5.8,H,1949-02-01,71,No smoker,Low risk consumption,U,U1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000085fdce1bb6d9e93904e0c632609e1c59085f,6.03,0.0,0.1250,2022-03-28,6.3,D,1930-08-01,91,No smoker,No risk consumption,R,NaN,1,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227312,fffe931b53eb160df80efc3d076c7a39c57b2780,6.17,0.3,0.0161,2011-05-16,6.3,D,1933-03-01,78,No smoker,No risk consumption,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227313,fffeae900480853f4b01e124674bbe67e283c62f,5.89,-0.1,0.0714,2023-04-26,6.0,D,1951-05-01,72,No smoker,No risk consumption,U,U2,1,1,NaN,NaN,NaN,NaN,NaN
227314,fffeffb3cbeb5a83dfb6a6e4cec485ff76fb79de,6.07,-0.2,0.0169,2022-09-14,6.0,H,1954-05-01,68,Smoker,Low risk consumption,R,NaN,NaN,1,NaN,NaN,NaN,1,NaN
227315,ffff8531e62f5d66787f60582f9d439732650d8e,6.13,-0.3,0.0164,2016-09-27,6.2,H,1934-03-01,82,Ex-smoker,No risk consumption,U,U4,1,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Adding column BMI from variables cliniques --> subset BMI generated in R
BMI = pyreadr.read_r("Data_preparation/data_BMI.rds")
data_BMI = next(iter(BMI.values()))

In [24]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_BMI,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='BMI'
)

Filtered out 0 negative values in 'BMI'.
Number of missing values in 'BMI': 11479


In [25]:
# Adding column Systolic blood pressure from variables cliniques --> subset SysBP generated in R
SysBP = pyreadr.read_r("Data_preparation/data_SysBP.rds")
data_SysBP = next(iter(SysBP.values()))

In [26]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_SysBP,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='systolic_blood_pressure'
)

Filtered out 0 negative values in 'systolic_blood_pressure'.
Number of missing values in 'systolic_blood_pressure': 2232


In [27]:
# Adding column Diastolic blood pressure from variables cliniques --> subset DiasBP generated in R
DiasBP = pyreadr.read_r("Data_preparation/data_DiasBP.rds")
data_DiasBP = next(iter(DiasBP.values()))

In [28]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_DiasBP,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='diastolic_blood_pressure'
)

Filtered out 0 negative values in 'diastolic_blood_pressure'.
Number of missing values in 'diastolic_blood_pressure': 2231


In [29]:
# Adding column cholesterol LDL from variables analitiques --> subset cLDL generated in R
cLDL = pyreadr.read_r("Data_preparation/data_cLDL.rds")
data_cLDL = next(iter(cLDL.values()))

In [30]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_cLDL,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='cholesterol_LDL'
)

Filtered out 0 negative values in 'cholesterol_LDL'.
Number of missing values in 'cholesterol_LDL': 5498


In [31]:
# Adding column triglycerides from variables analitiques --> subset trigly generated in R
trigly = pyreadr.read_r("Data_preparation/data_trigly.rds")
data_trigly = next(iter(trigly.values()))

In [32]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_trigly,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='triglycerides'
)

Filtered out 127 negative values in 'triglycerides'.
Number of missing values in 'triglycerides': 1305


In [33]:
# Adding column eGFR from variables analitiques --> subset eGFR generated in R
eGFR = pyreadr.read_r("Data_preparation/data_eGFR.rds")
data_eGFR = next(iter(eGFR.values()))

In [34]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_eGFR,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='eGFR'
)

Filtered out 0 negative values in 'eGFR'.
Number of missing values in 'eGFR': 1999


In [35]:
# Adding column ACR from variables cliniques --> subset ACR generated in R
ACR = pyreadr.read_r("Data_preparation/data_ACR.rds")
data_ACR = next(iter(ACR.values()))

In [36]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_ACR,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='ACR'
)

Filtered out 16604 negative values in 'ACR'.
Number of missing values in 'ACR': 72230


In [37]:
display(input_data_trend)

,idp,value_mean,delta_last,relative_change,dat,val_HbA1c,sexe,dnaix,age,smoking_status,...,stroke,ischemic_heart_disease,chronic_kidney_disease,BMI,systolic_blood_pressure,diastolic_blood_pressure,cholesterol_LDL,triglycerides,eGFR,ACR
0,0000072333d55312fe62d5b5458a22efa4257e27,5.97,0.7,0.1607,2023-04-27,6.5,D,1949-09-01,73,No smoker,...,NaN,NaN,NaN,25.78,126.0,68.0,97.0,105.0,60.1,35.94
1,0000150454aa8670459408f45e453242f7bcde9c,5.80,0.4,0.0517,2023-02-15,6.1,H,1962-02-01,61,No smoker,...,1,NaN,NaN,32.03,133.0,87.0,47.0,166.0,60.1,2.46
2,0000214c9856e03de46a8f927fc1e08b7c943578,6.04,-0.3,0.0000,2022-12-07,5.9,H,1947-06-01,75,No smoker,...,NaN,1,NaN,23.38,133.0,81.0,65.0,83.0,60.1,2.21
3,000058a85618eb0c0d59f3e9056372e87e5d426e,5.40,0.6,0.0943,2020-10-30,5.8,H,1949-02-01,71,No smoker,...,NaN,NaN,NaN,25.72,137.0,75.0,92.0,120.0,60.1,1.00
4,000085fdce1bb6d9e93904e0c632609e1c59085f,6.03,0.0,0.1250,2022-03-28,6.3,D,1930-08-01,91,No smoker,...,NaN,NaN,1,24.55,132.0,75.0,81.0,89.0,47.8,20.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227312,fffe931b53eb160df80efc3d076c7a39c57b2780,6.17,0.3,0.0161,2011-05-16,6.3,D,1933-03-01,78,No smoker,...,NaN,NaN,NaN,28.13,150.0,75.0,131.0,85.0,60.1,10.70
227313,fffeae900480853f4b01e124674bbe67e283c62f,5.89,-0.1,0.0714,2023-04-26,6.0,D,1951-05-01,72,No smoker,...,NaN,NaN,NaN,26.86,149.0,81.0,169.0,195.0,60.1,3.37
227314,fffeffb3cbeb5a83dfb6a6e4cec485ff76fb79de,6.07,-0.2,0.0169,2022-09-14,6.0,H,1954-05-01,68,Smoker,...,NaN,1,NaN,28.50,132.0,79.0,65.0,318.0,60.1,7.60
227315,ffff8531e62f5d66787f60582f9d439732650d8e,6.13,-0.3,0.0164,2016-09-27,6.2,H,1934-03-01,82,Ex-smoker,...,NaN,NaN,NaN,29.30,140.0,70.0,75.0,110.0,60.1,26.30


In [38]:
# Save generated input_data_trend-file
input_data_trend.to_csv("generated_input_data_trend_NEW.csv", index=False)